In [13]:
import pandas as pd
from urllib import parse

In [5]:
ad_stats = pd.read_excel('ad_campaigns.xlsx')

In [6]:
ad_stats.rename(columns = {'Название группы': 'group', 'Фраза (с минус-словами)': 'phrase', 'Продуктивность': 'effect',
                         'ID объявления': 'ad_id', 'Заголовок': 'title', 'Текст': 'text', 'Ссылка': 'link'}, inplace=True)

In [8]:
url = ad_stats.loc[0, 'link']

In [9]:
parsed = parse.urlsplit(url)

In [10]:
query_string = parsed.query

In [15]:
params_dict = parse.parse_qs(query_string)

In [17]:
def company_name(row):
    """
    Поллучение названия компании из ссылки внутри 
    row.
    """
    parsed = parse.urlsplit(row['link'])
    params_dict = parse.parse_qs(parsed.query)
    return params_dict['utm_campaign'][0]

In [18]:
ad_stats['campaign'] = ad_stats.apply(company_name, axis=1)
ad_stats.head()

,group,phrase,effect,ad_id,title,text,link,campaign
0,мрт менделеевская,"""!мрт !менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87
1,мрт цао,"""мрт менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87
2,мрт цао,мрт менделеевская -головы -позвоночника -сдела...,7.4,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87
3,мрт цао,"""!мрт !цао""",5.0,2101704999,МРТ в ЦАО от 2000₽. Звоните!,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,9cc0c5f6ca
4,мрт сао,"""мрт цао""",5.0,2101704999,МРТ в ЦАО от 2000₽. Звоните!,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,9cc0c5f6ca


In [19]:
def effect_class(row):
    if row['effect'] <= 3:
        return 'Низкий'
    elif row['effect'] <= 7:
        return 'Средний'
    
    return 'Высокий'

In [20]:
ad_stats['effect_class'] = ad_stats.apply(effect_class, axis=1)

In [21]:
ad_stats.head()

,group,phrase,effect,ad_id,title,text,link,campaign,effect_class
0,мрт менделеевская,"""!мрт !менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87,Средний
1,мрт цао,"""мрт менделеевская""",4.5,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87,Средний
2,мрт цао,мрт менделеевская -головы -позвоночника -сдела...,7.4,2101704995,МРТ на Менделеевской от 2000₽,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,a825749b87,Высокий
3,мрт цао,"""!мрт !цао""",5.0,2101704999,МРТ в ЦАО от 2000₽. Звоните!,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,9cc0c5f6ca,Средний
4,мрт сао,"""мрт цао""",5.0,2101704999,МРТ в ЦАО от 2000₽. Звоните!,24 часа. МРТ в день обращения. Консультация вр...,https://awesome-site.ru/?utm_source=yandex&utm...,9cc0c5f6ca,Средний


In [28]:
ad_stats.pivot_table(index='campaign', columns='effect_class', values='effect', 
                     aggfunc=sum, fill_value=0)

effect_class,Высокий,Низкий,Средний
campaign,,,
034ada41b6,17.5,0.0,4.1
03fcaecd1d,0.0,0.0,10.5
0697a81555,0.0,0.0,9.5
08cdcb57a3,7.4,0.0,11.0
0f0ba311fb,0.0,0.0,10.8
...,...,...,...
f6d2ae1e3d,69.6,384.5,301.2
f80e902d76,0.0,0.0,3.3
fbed7afb58,8.6,0.0,7.6
